## Function which returns topology/positions without selected residue
* The indices must include all atoms of the residue to be removed 
* i.e. [atom.index for atom in ala_topology.atoms() if atom.residue.name == 'ALA']

In [11]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *

from openmmtools.testsystems import AlanineDipeptideExplicit

In [12]:
# Using Alanine dipeptide system
alanine = AlanineDipeptideExplicit()
ala_system, ala_positions, ala_topology = alanine.system, alanine.positions,  alanine.topology
ala_residue = [residue for residue in ala_topology.residues() if residue.name == 'ALA']

In [13]:
forcefield = ForceField('amber99sb.xml','tip3p.xml')

In [21]:
def remove_residue_top(topology,replace_residue):
    """Copy topology information while excluding a selected residue"""
    
    assert len(replace_residue) == 1, 'Must specify a single residue'
    
    # Create empty topology
    new_topology = Topology()
    
    for chain in topology.chains():
        new_chain = new_topology.addChain(chain)
        for residue in chain.residues():      
            if residue == replace_residue[0]:
                # skip residue
                pass
            else:
                new_residue = new_topology.addResidue(residue.name,new_chain)
                for atom in residue.atoms():
                    new_topology.addAtom(atom.name, atom.element, new_residue)
    
    # Create standard bonds
    new_topology.createStandardBonds()
        
    return new_topology

In [22]:
def remove_residue_pos(topology,positions,replace_residue):
    """Copy positions while excluding select atomic indices (replace_indices)"""
    
    residue_atoms = [atom.index for atom in replace_residue[0].atoms()]
    assert len(residue_atoms) > 0, 'Residue indicies empty is empty'
    
    # Create empty list 
    new_positions = Quantity(unit=nanometer)
    
    # Append positions to new position list
    for position, atom in zip(positions,topology.atoms()):
        if atom.index in residue_atoms:
            pass
        else:
            new_positions.append(position)
            
    return new_positions

In [23]:
topology = remove_residue_top(ala_topology,ala_residue)
positions = remove_residue_pos(ala_topology,ala_positions,ala_residue)

In [26]:
topology

<Topology; 1 chains, 751 residues, 2259 atoms, 1509 bonds>

In [8]:
# testing if the system is stable
system = forcefield.createSystem(topology)
integrator = LangevinIntegrator(300*kelvin,1/picoseconds,0.001*picoseconds)

simulation = Simulation(topology,system,integrator)

simulation.context.setPositions(positions)

In [9]:
simulation.minimizeEnergy()

In [75]:
simulation.reporters.append(PDBReporter('stability_test.pdb',100))

In [76]:
simulation.step(1000)

### This can be done far more easily by writing a PDB file and remove the residue i.e. through MDtraj

In [10]:
PDBFile.writeFile(ala_topology,ala_positions,open('ACE-ALA-NME_explicit.pdb','w'))

In [11]:
import mdtraj as md

In [12]:
pdb = md.load_pdb('ACE-ALA-NME_explicit.pdb')

In [13]:
md_alanine = [atom.index for atom in pdb.topology.atoms if atom.residue.name != 'ALA']

In [14]:
new_pdb = pdb.atom_slice(md_alanine)

In [15]:
new_pdb.save('ACE-NME_explicit.pdb')